In [1]:
import numpy as np
from scipy.sparse import csr_matrix, hstack
from scipy.stats import norm


def fix_age(x, headers):
    ix_age = np.nonzero([h.startswith('demographics_age') for h in headers])[0]
    print('AGE IX is:', ix_age)
    xage = x[:, ix_age].toarray()
    print('turning age into bins: (original x shape:)', x.shape)
    bins = [1, 5, 10, 20, 30, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 150]
    xagebins = np.hstack([np.array((xage >= bins[i]) & (xage < bins[i+1])) for i in range(0, len(bins)-1)])
    headersnew = np.array(['demographics_age[between_'+str(bins[i])+'_and_'+str(bins[i+1])+']'
                           for i in range(0, len(bins)-1)])
    x = hstack([x, csr_matrix(xagebins)], format='csr')
    headers = np.hstack([headers, headersnew])
    print('done. x is now of shape:', x.shape)
    return x, headers


def fix_binary(x, headers):
    print('fixing diagnosis and procedure columns and adding bins: (original x shape:)', x.shape) 
    diag_ix = np.nonzero([h.startswith("Diagnosis:") or h.startswith("ProcedureCPT") or h.startswith("Medication")
                          for h in headers])[0]
    bins = [0, 2, 10]
    xbin = hstack([(x[:, diag_ix] > bins[i]) for i in range(len(bins))], format='csr')
    headersbin = np.hstack([[h+'[count>'+str(bins[i])+']' for h in headers[diag_ix]] for i in range(len(bins))])
    x = hstack([xbin, x], format='csr')
    headers = np.hstack([headersbin, headers])
    print('done. x is now of shape:', x.shape)
    return x, headers


def normalize_csr_matrix(x, headers, meanarr=None, stdarr=None, ixnormed=None, threshold_to_clip=2000, epsilon=0.0001):
    xnorm = x.copy()
    if meanarr is None and stdarr is None and ixnormed is None:
        nnz_cnt_denominator = np.array((x != 0).sum(axis=0)).ravel()
        nnz_columns_ix = (nnz_cnt_denominator == 0).ravel()
        nnz_cnt_denominator[nnz_columns_ix] = 1.0 
        nnz_mean_sum = np.array(x.sum(axis=0)).ravel()
        mean_array = nnz_mean_sum/nnz_cnt_denominator
        nnz_std_sum_part1 = np.array((x.multiply(x)).sum(axis=0)).ravel()
        std_array = np.sqrt((nnz_std_sum_part1/nnz_cnt_denominator) - (mean_array ** 2))
        ix_to_normalize = (std_array != 0) & (std_array < threshold_to_clip) & (np.array([hi.startswith('Lab') or hi == 'demographics_age' for hi in headers]))
        ix_to_not_normalize = (True ^ ix_to_normalize).nonzero()[0].ravel()
        mean_array[ix_to_not_normalize] = 0.0
        std_array[ix_to_not_normalize] = 1.0
    else:
        print('mean and sd already specified. normalizing with given mean/sd.')
        mean_array = meanarr
        std_array = stdarr
        ix_to_normalize = np.zeros(std_array.shape, dtype=bool)
        ix_to_normalize[ixnormed] = True
    # normalize the nonzero values: Note that we won't eliminate_zeros() here to keep the values that were equal
    # to the mean available for next steps.
    xnorm.data = (xnorm.data - mean_array[xnorm.indices])/std_array[xnorm.indices]
    return xnorm, mean_array, std_array, ix_to_normalize.nonzero()[0].ravel()


def pick_bin_columns(x, headers, ix_to_pick=None):
    print('keeping only binary features (plus age): (original x shape:)', x.shape)
    ix_age = np.nonzero([h == 'demographics_age' for h in headers])[0]
    print('AGE IX is:', ix_age)
    if ix_to_pick is None:
        bin_ix = np.nonzero(((x.max(axis=0)).toarray().ravel() == 1).ravel() & ((x.min(axis=0)).toarray().ravel() == 0).ravel())[0]
        bin_ix = np.union1d(bin_ix, ix_age)  # we will keep the age. At this point it is already normalized
    else:
        bin_ix = ix_to_pick
    x = x[:, bin_ix]
    headers = headers[bin_ix]
    print('done. x is now of shape:', x.shape)
    return x, headers, bin_ix


def pick_freq_columns(x, headers, minfreq=100, source_freq_ix=None):
    if source_freq_ix is None:
        print((f'keeping only frequent columns that are observed at least in: {minfreq} rows: '
               f'(original x shape: {x.shape}'))
        freq_ix = np.nonzero(np.array((x != 0).sum(axis=0)).ravel() > minfreq)[0]
    else:
        freq_ix = source_freq_ix
    print('total num of frequent columns:', len(freq_ix))
    x = x[:, freq_ix]
    headers = headers[freq_ix]
    print('done. x is now of shape:', x.shape)
    return x, headers, freq_ix


# def load_preprocessed_data(datadir):
#     (x, y, ylabel, headers, ixtrain, ixtest, mrns) = load_data(datadir)
#     new_headers = pickle.load(open(datadir + '/new_headers.pkl', 'rb'))
#     newx = pickle.load(open(datadir + '/new_x.pkl', 'rb'))
#     return x, y, ylabel, headers, ixtrain, ixtest, mrns, newx, new_headers


def get_or_rr_pval(x, y, headers):
    print('computing pvalues')
 
    ylabel = y.reshape(-1, 1)
    ix_total_pos = np.nonzero(ylabel != 0)[0].ravel()
    ix_total_neg = np.nonzero(ylabel == 0)[0].ravel()
    total_neg_rows = ix_total_neg.shape[0]
    total_pos_rows = ix_total_pos.shape[0]

    bin_headers = (x.max(axis=0) == 1).toarray().ravel() & (x.min(axis=0) == 0).toarray().ravel()
    non_bin_headers = (True ^ bin_headers) & (np.array(abs(x).sum(axis=0)).ravel() != 0)
    print('binary headers total:', bin_headers.shape, 'non binary headers total:', non_bin_headers.shape)

    de = np.array((x[ix_total_pos, :] != 0).sum(axis=0)).ravel()  # np.array((x != 0).multiply(ylabel > 0).sum(axis=0)).ravel() ;
    he = np.array((x[ix_total_neg, :] != 0).sum(axis=0)).ravel()  # np.array((x != 0).multiply(ylabel == 0).sum(axis=0)).ravel();
    dn = total_pos_rows - np.array((x[ix_total_pos, :] != 0).sum(axis=0)).ravel()  # np.array((x == 0).multiply(ylabel > 0).sum(axis=0)).ravel();
    hn = total_neg_rows - np.array((x[ix_total_neg, :] != 0).sum(axis=0)).ravel()  # np.array((x == 0).multiply(ylabel == 0).sum(axis=0)).ravel();

    reliable_ix_bin = (de != 0) & (he != 0) & (dn != 0) & (hn != 0) & bin_headers

    or_ = (de/he)/(dn/hn)
    or_ste = np.sqrt(1/de + 1/he + 1/dn + 1/hn)
    or_low, or_high = np.exp(np.log(or_) - 1.96*or_ste), np.exp(np.log(or_) + 1.96*or_ste)
    rr = (de/(de+he))/(dn/(dn+hn))
    or_[non_bin_headers] = 0
    or_ste[non_bin_headers] = 0
    or_low[non_bin_headers] = 0
    or_high[non_bin_headers] = 0
    rr[non_bin_headers] = 0
    pvalue_bin = 2 * norm.cdf(-1*(np.abs(np.log(or_))/or_ste))
    pvalue_bin[True ^ reliable_ix_bin] = np.nan

    xposnormed, means_pos, sd_pos, ix_pos_normed = normalize_csr_matrix(x[ix_total_pos, :], headers)
    xnegnormed, means_neg, sd_neg, ix_neg_normed = normalize_csr_matrix(x[ix_total_neg, :], headers)
    ix_pos_normed_array = np.array([True if i in ix_pos_normed else False for i in range(len(headers))])
    ix_neg_normed_array = np.array([True if i in ix_neg_normed else False for i in range(len(headers))])
    mds = means_pos - means_neg
    var_pos = sd_pos ** 2
    nnz_pos = np.array((x[ix_total_pos, :] != 0).sum(axis=0)).ravel()
    var_neg = sd_neg ** 2
    nnz_neg = np.array((x[ix_total_neg, :] != 0).sum(axis=0)).ravel()
    ste = np.sqrt(var_pos/nnz_pos) + np.sqrt(var_neg/nnz_neg)
    # LCL, UCL = MDs - 2*STE , MDs + 2*STE
    z = mds/ste
    reliable_ix_con = (non_bin_headers) & (ste != 0) & (ix_pos_normed_array == True) & (ix_neg_normed_array == True)
    pvalue_cont = (2 * norm.cdf(-np.abs(z))).ravel()
    pvalue_cont[True ^ reliable_ix_con] = np.nan


def csr_mask_in_range(x, binlow, binhigh):
    xmask = x.copy()
    in_range_ix = (xmask.data >= binlow) & (xmask.data < binhigh)
    non_in_range_ix = True ^ in_range_ix
    xmask.data[in_range_ix] = 1
    xmask.data[non_in_range_ix] = 0
    xmask.eliminate_zeros()
    return xmask


def augment_data_with_std_cats(x, headers, meanarr=None, stdarr=None, ixnormed=None):
    print('normalizing lab value columns. (original x shape:)', x.shape)
    if meanarr is None and stdarr is None:
        x, mean_array, std_array, ix_to_normalize = normalize_csr_matrix(x, headers)
    else:
        x, mean_array, std_array, ix_to_normalize = normalize_csr_matrix(x, headers, meanarr, stdarr, ixnormed)

    nonbin_headers = headers[ix_to_normalize]
    x2 = x[:, ix_to_normalize]
    bins = [-10, -3, -1, -0.5, 0.5, 1, 3, 10]
    print('augmenting those columns with dynamic bins')
    headersnew = np.array([hi + '[normedValue_sd_between_' + str(bins[i]) + ',' + str(bins[i+1]) + ']'
                           for i in range(0, len(bins)-1) for hi in nonbin_headers])
    xlabsbins = []
    for i in range(len(bins)-1):
        binlow, binhigh = bins[i], bins[i+1]
        # slow way:
        # if (binlow < 0):
        #     binlowMX = (x2 >= binlow)
        # else:
        #     binlowMX = (x2 >= binlow)
        # if (binhigh > 0):
        #     binhighMX = (x2 < binhigh) 
        # else:
        #     binhighMX = (x2 < binhigh)
        # xlabsbins.append( (x2 != 0).multiply(binlowMX).multiply(binhighMX) )
        # fast way:
        xlabsbins.append(csr_mask_in_range(x2, binlow, binhigh))
    xlabsbins = hstack(xlabsbins, format='csr')
    print('stacking the bins to new data')
    x = hstack([xlabsbins, x], format='csr')
    headers = np.hstack([headersnew, headers])
    print('done adding bins for labs. x is now of shape:', x.shape)
    return x, headers, mean_array, std_array, ix_to_normalize


def pre_process_data(x_outcome, headers, source_preprocess_vars=None):
    if source_preprocess_vars is None:
        freq_ix = None
        meanarr, stdarr, ixnormed = None, None, None
        ix_to_pick = None
    else:
        freq_ix = source_preprocess_vars['freq_ix']
        meanarr = source_preprocess_vars['meanarr']
        stdarr = source_preprocess_vars['stdarr']
        ixnormed = source_preprocess_vars['ixnormed']
        ix_to_pick = source_preprocess_vars['ix_to_pick']

    x_outcome, headers = fix_age(x_outcome, headers)
    x_outcome, headers, freq_ix = pick_freq_columns(x_outcome, headers, 100, freq_ix)
    x_outcome, headers, mean_array, std_array, ix_to_normalize = augment_data_with_std_cats(x_outcome, headers, meanarr,
                                                                                            stdarr, ixnormed)
    x_outcome, headers = fix_binary(x_outcome, headers)
    x_outcome, headers, ix_to_pick = pick_bin_columns(x_outcome, headers, ix_to_pick)

    preprocess_vars = {'freq_ix': freq_ix, 'meanarr': mean_array, 'stdarr': std_array,
                       'ixnormed': ix_to_normalize, 'ix_to_pick': ix_to_pick}
    return x_outcome, headers, preprocess_vars


In [2]:
import pickle 

headerFile = "/gpfs/data/razavianlab/capstone/2021_ehr/headers.pkl"
mrnsFile = "/gpfs/data/razavianlab/capstone/2021_ehr/mrns_all_deid.pkl"
xallFile = "/gpfs/data/razavianlab/capstone/2021_ehr/xall.pkl"

In [3]:
headerArr = np.load(headerFile, allow_pickle=True)
with open(xallFile, 'rb') as f:
    data = pickle.load(f)

In [4]:
new_xall, headers, processed_vars = pre_process_data(data, headerArr)

AGE IX is: [193517]
turning age into bins: (original x shape:) (11044215, 470305)
done. x is now of shape: (11044215, 470323)
keeping only frequent columns that are observed at least in: 100 rows: (original x shape: (11044215, 470323)
total num of frequent columns: 66914
done. x is now of shape: (11044215, 66914)
normalizing lab value columns. (original x shape:) (11044215, 66914)
augmenting those columns with dynamic bins
stacking the bins to new data
done adding bins for labs. x is now of shape: (11044215, 71828)
fixing diagnosis and procedure columns and adding bins: (original x shape:) (11044215, 71828)
done. x is now of shape: (11044215, 270245)
keeping only binary features (plus age): (original x shape:) (11044215, 270245)
AGE IX is: [255298]
done. x is now of shape: (11044215, 192330)


In [152]:
# function which removes specified row and col indices from csr matrix

def delete_from_csr(mat, row_indices=[], col_indices=[]):
    rows = []
    cols = []
    if row_indices:
        rows = list(row_indices)
    if col_indices:
        cols = list(col_indices)

    if len(rows) > 0 and len(cols) > 0:
        row_mask = np.ones(mat.shape[0], dtype=bool)
        row_mask[rows] = False
        col_mask = np.ones(mat.shape[1], dtype=bool)
        col_mask[cols] = False
        return mat[row_mask][:,col_mask]
    elif len(rows) > 0:
        mask = np.ones(mat.shape[0], dtype=bool)
        mask[rows] = False
        return mat[mask]
    elif len(cols) > 0:
        mask = np.ones(mat.shape[1], dtype=bool)
        mask[cols] = False
        return mat[:,mask]
    else:
        return mat

In [153]:
print ('before deleting age_demographics')
print (new_xall.shape)

before deleting age_demographics
(11044215, 192330)


In [154]:
# delete SIM columns from data
age_demographics_index = 191846
new_xall_v2 = delete_from_csr(new_xall, row_indices=[], col_indices=[age_demographics_index])

In [155]:
print ('after deleting age_demographics')
print (new_xall_v2.shape)

after deleting age_demographics
(11044215, 192329)


In [156]:
# update headerArr with updated indices

headers_v2 = np.delete(headers, [age_demographics_index])

In [172]:
# example of a particular patient with their present medical codes

patient_study_index = 1545

In [173]:
new_xall_v2.data[new_xall_v2.indptr[patient_study_index]:new_xall_v2.indptr[patient_study_index+1]]

array([1., 1., 1., ..., 1., 1., 1.])

In [174]:
# find all column indicies of patient in study which are non-zero
# potential input to BERT (mapping indices to strings and separating by spaces)

new_xall_v2.indices[new_xall_v2.indptr[patient_study_index]:new_xall_v2.indptr[patient_study_index+1]]

array([192322, 191844, 191838, ...,    350,    349,    270], dtype=int32)

In [175]:
# find all column names of patient in study which are non-zero
# potential input to BERT (with spaces removed and separating column names by spaces)

headers[list(new_xall_v2.indices[new_xall_v2.indptr[patient_study_index]:new_xall_v2.indptr[patient_study_index+1]])]

array(['demographics_age[between_65_and_70]', 'demographics_GENDER_Male',
       'demographics_RACE_Other Race', ...,
       'Diagnosis:Malignant neoplasm of mandible C41.1[count>0]',
       'Diagnosis:Malignant neoplasm of bones of skull and face C41.0[count>0]',
       'Diagnosis:Malignant neoplasm of mouth, unspecified C06.9[count>0]'],
      dtype='<U301')

### Saving Preprocessed Data

In [ ]:
# saving new data and header files
scipy.sparse.save_npz('preprocessed_xall.npz', new_xall_v2)

In [183]:
with open('preprocessed_headers.pkl','wb') as f:
    pickle.dump(headers_v2, f)

In [ ]:
np.save(new)

In [23]:
headerLength = headers.shape[0]

colonSplit = np.char.split(headers, sep = ':')
# initializing prefix storage
raw_prefixes = np.empty(shape=(headerLength), dtype='<U301')  

# getting first string (string before colon)
for i in range(colonSplit.shape[0]):
    raw_prefixes[i] = colonSplit[i][0]

prefixes = np.unique(raw_prefixes, return_counts=True)

In [140]:
list(headers).index('demographics_age')

191846

In [72]:
csc_data = scipy.sparse.csc_matrix(new_xall)

In [ ]:
def fix_binary(x, headers):
    print('fixing diagnosis and procedure columns and adding bins: (original x shape:)', x.shape) 
    diag_ix = np.nonzero([h.startswith("Diagnosis:") or h.startswith("ProcedureCPT") or h.startswith("Medication")
                          for h in headers])[0]
    bins = [0, 2, 10]
    xbin = hstack([(x[:, diag_ix] > bins[i]) for i in range(len(bins))], format='csr')
    headersbin = np.hstack([[h+'[count>'+str(bins[i])+']' for h in headers[diag_ix]] for i in range(len(bins))])
    x = hstack([xbin, x], format='csr')
    headers = np.hstack([headersbin, headers])
    print('done. x is now of shape:', x.shape)
    return x, headers

In [40]:
new_xall.count_nonzero()

939629263

In [41]:
939629263/(new_xall.shape[0]*new_xall.shape[1])

0.0004423587777825694